In [195]:
import requests
import pandas as pd
from datetime import datetime
import json
import os
import glob
import ast

# Part 1: Data Retrieval

## Step 0: Set parameters

In [174]:
API_KEY = "oe_3ZnNrr7V85CXb5QUrsTWhXq1"  # API key for authentication
BASE_URL = "https://api.openelectricity.org.au"  # Base URL for the Open Electricity API

Facility = "/v4/facilities/"  # Endpoint for retrieving facility information
Facility_data = "/v4/data/facilities/"  # Endpoint for retrieving facility data
NETWORK_ID = "NEM"  # Network ID (NEM: National Electricity Market)

interval = "5m"  # Time interval for data retrieval (5 minutes)
date_start = datetime(2025, 10, 1, 0, 0, 0)  # Start date for data collection
date_end = end_date = datetime(2025, 10, 7, 23, 59, 59)  # End date for data collection

# URL for generation (facility-level) data API
# Reference: https://docs.openelectricity.org.au/api-reference/data/get-facility-data
url_Generation = BASE_URL + Facility_data + NETWORK_ID

# URL for market-level data API
# Reference: https://docs.openelectricity.org.au/api-reference/market/get-network-data
url_market = BASE_URL + "/v4/market/network/" + NETWORK_ID

# Retrieve all .csv files in the current working directory
csv_files = glob.glob("*.csv")

# Iterate through the list and delete each CSV file
for file in csv_files:
    try:
        os.remove(file)
        print(f"Deleted: {file}")  # Print confirmation of deletion
    except Exception as e:
        print(f"Error deleting {file}: {e}")  # Print any errors encountered during deletion


Get all facilities which are currently on operation in NEM.

In [175]:
# Construct the full API URL for retrieving facility information
url = BASE_URL + Facility

# Define request headers with authorization token
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# Define query parameters for the API request
params = {
    "network_id": ["NEM"],  # Target the NEM network
    "with_clerk": True      # Include detailed facility information
}

# Send GET request to the API
response = requests.get(url, headers=headers, params=params)

# Initialize a list to store operating facilities only
operating_facilities = []

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if data.get("success"):
        facilities = data.get("data", [])

        if facilities:
            # Save the full facility data to CSV
            df = pd.json_normalize(facilities)
            df.to_csv("nem_operating_facilities.csv", index=False)
            print(f"Saved {len(df)} NEM facilities to nem_facilities.csv")

            # Filter out only facilities that have at least one operating unit
            for f in facilities:
                units = f.get("units", [])
                if any(u.get("status_id") == "operating" for u in units):
                    operating_facilities.append({"code": f.get("code"), "name": f.get("name")})

            # Print summary information
            print(f"Found {len(operating_facilities)} operating facilities")
            print("List of operating facilities (code & name):")
            print(operating_facilities)

        else:
            print("No facilities found.")
    else:
        print("API returned error:", data.get("error"))
else:
    print("HTTP error:", response.status_code, response.text)

Saved 514 NEM facilities to nem_facilities.csv
Found 419 operating facilities
List of operating facilities (code & name):
[{'code': 'ADP', 'name': 'Adelaide Desalination'}, {'code': 'ALDGASF', 'name': 'Aldoga'}, {'code': 'ANGASTON', 'name': 'Angaston'}, {'code': 'APPIN', 'name': 'Appin'}, {'code': 'ARWF', 'name': 'Ararat'}, {'code': 'AVLSF', 'name': 'Avonlie'}, {'code': 'DEIBDL', 'name': 'Bairnsdale'}, {'code': 'BAKING', 'name': 'Baking Board'}, {'code': 'BHWF', 'name': 'Bald Hills'}, {'code': 'BALBESS', 'name': 'Ballarat'}, {'code': 'BANGOWF', 'name': 'Bango'}, {'code': 'BAPS', 'name': 'Banimboola'}, {'code': 'BANNSP', 'name': 'Bannerton'}, {'code': 'BARCALDN', 'name': 'Barcaldine'}, {'code': 'BARCSF', 'name': 'Barcaldine'}, {'code': 'BARKIPS', 'name': 'Barker Inlet'}, {'code': 'BARRON', 'name': 'Barron Gorge'}, {'code': 'BASTYAN', 'name': 'Bastyan'}, {'code': 'BAYSW', 'name': 'Bayswater'}, {'code': 'BBP31', 'name': 'Bell Bay Three'}, {'code': 'BRYB1WF1', 'name': 'Berrybank'}, {'code'

## Step 1: Fetch Generation Information for each facility in NEM 

In [176]:
def fetch_facility_metric(url, metrics, interval, date_start, date_end, fcode, print_mode=False, save_csv=False):
    """
    Fetch data for a specific facility and optionally save it as a CSV file.

    Parameters:
        url (str): API endpoint for facility data
        metrics (str or list): Metric(s) to fetch (e.g., "power", "emissions")
        interval (str): Data interval (e.g., "5m", "1h")
        date_start (str): Start datetime in ISO format
        date_end (str): End datetime in ISO format
        fcode (str): Facility code
        print_mode (bool): If True, print raw API data
        save_csv (bool): If True, save the result to a CSV file

    Returns:
        pd.DataFrame: A DataFrame containing facility data
    """

    # Set request headers with API key
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    # Set query parameters for API request
    params = {
        "metrics": metrics,
        "interval": interval,
        "date_start": date_start,
        "date_end": date_end,
        "facility_code": fcode
    }

    # Make GET request
    response = requests.get(url, headers=headers, params=params)

    # Handle failed request
    if response.status_code != 200:
        print(f"Failed, status code: {response.status_code}")
        print(response.text)
        return pd.DataFrame() 

    # Extract data from JSON response
    data = response.json().get("data", [])
    if print_mode:
        print(data)
        
    # Return None if no data returned
    if not data:
        print("No data returned.")
        return None

    # Flatten data into rows
    rows = []
    for metric_block in data:
        results = metric_block.get("results", [])
        for unit in results:
            unit_code = unit["columns"]["unit_code"]
            metric_name = unit["name"]
            for entry in unit["data"]:
                timestamp, value = entry
                rows.append({
                    "facility_code": fcode,
                    "unit_code": unit_code,
                    "metric": metric_name,
                    "timestamp": timestamp,
                    "value": value
                })

    # Convert to DataFrame
    df = pd.DataFrame(rows)

    # Optionally save to CSV
    if save_csv:
        csv_filename = f"{fcode}_{metrics}_{interval}.csv"
        df.to_csv(csv_filename, index=False)
        print(f"Saved {len(df)} rows to {csv_filename}")

    return df

In [177]:
# Test the function for "power"
fetch_facility_metric(BASE_URL + Facility_data + NETWORK_ID, "power", interval, date_start.isoformat(), date_end.isoformat(), "ADP", print_mode=True)

[{'network_code': 'NEM', 'metric': 'power', 'unit': 'MW', 'interval': '5m', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'groupings': [], 'results': [{'name': 'power_ADPBA1', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'columns': {'unit_code': 'ADPBA1'}, 'data': [['2025-10-01T00:00:00+10:00', -0.004], ['2025-10-01T00:05:00+10:00', -0.046], ['2025-10-01T00:10:00+10:00', 0.0], ['2025-10-01T00:15:00+10:00', 0.003], ['2025-10-01T00:20:00+10:00', -0.018], ['2025-10-01T00:25:00+10:00', 0.004], ['2025-10-01T00:30:00+10:00', -0.001], ['2025-10-01T00:35:00+10:00', -0.003], ['2025-10-01T00:40:00+10:00', 0.076], ['2025-10-01T00:45:00+10:00', -0.001], ['2025-10-01T00:50:00+10:00', 0.058], ['2025-10-01T00:55:00+10:00', -0.017], ['2025-10-01T01:00:00+10:00', 0.005], ['2025-10-01T01:05:00+10:00', 0.027], ['2025-10-01T01:10:00+10:00', 0.003], ['2025-10-01T01:15:00+10:00', 0.0], ['2025-10-01T01:20:00+10:00', 0.0], ['2025-10-

,facility_code,unit_code,metric,timestamp,value
0,ADP,ADPBA1,power_ADPBA1,2025-10-01T00:00:00+10:00,-0.004
1,ADP,ADPBA1,power_ADPBA1,2025-10-01T00:05:00+10:00,-0.046
2,ADP,ADPBA1,power_ADPBA1,2025-10-01T00:10:00+10:00,0.000
3,ADP,ADPBA1,power_ADPBA1,2025-10-01T00:15:00+10:00,0.003
4,ADP,ADPBA1,power_ADPBA1,2025-10-01T00:20:00+10:00,-0.018
...,...,...,...,...,...
6043,ADP,ADPBA1L,power_ADPBA1L,2025-10-07T21:20:00+10:00,0.058
6044,ADP,ADPBA1L,power_ADPBA1L,2025-10-07T21:45:00+10:00,0.118
6045,ADP,ADPBA1L,power_ADPBA1L,2025-10-07T21:50:00+10:00,0.008
6046,ADP,ADPBA1L,power_ADPBA1L,2025-10-07T22:00:00+10:00,0.033


In [178]:
# Test the function for "emissions"
fetch_facility_metric(BASE_URL + Facility_data + NETWORK_ID, "emissions", interval, date_start.isoformat(), date_end.isoformat(), "ADP", print_mode=True)

[{'network_code': 'NEM', 'metric': 'emissions', 'unit': 't', 'interval': '5m', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'groupings': [], 'results': [{'name': 'emissions_ADPBA1', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'columns': {'unit_code': 'ADPBA1'}, 'data': [['2025-10-01T00:00:00+10:00', 0.0], ['2025-10-01T00:05:00+10:00', 0.0], ['2025-10-01T00:10:00+10:00', 0.0], ['2025-10-01T00:15:00+10:00', 0.0], ['2025-10-01T00:20:00+10:00', 0.0], ['2025-10-01T00:25:00+10:00', 0.0], ['2025-10-01T00:30:00+10:00', 0.0], ['2025-10-01T00:35:00+10:00', 0.0], ['2025-10-01T00:40:00+10:00', 0.0], ['2025-10-01T00:45:00+10:00', 0.0], ['2025-10-01T00:50:00+10:00', 0.0], ['2025-10-01T00:55:00+10:00', 0.0], ['2025-10-01T01:00:00+10:00', 0.0], ['2025-10-01T01:05:00+10:00', 0.0], ['2025-10-01T01:10:00+10:00', 0.0], ['2025-10-01T01:15:00+10:00', 0.0], ['2025-10-01T01:20:00+10:00', 0.0], ['2025-10-01T01:25:00+10:00', 0.0], ['

,facility_code,unit_code,metric,timestamp,value
0,ADP,ADPBA1,emissions_ADPBA1,2025-10-01T00:00:00+10:00,0.0
1,ADP,ADPBA1,emissions_ADPBA1,2025-10-01T00:05:00+10:00,0.0
2,ADP,ADPBA1,emissions_ADPBA1,2025-10-01T00:10:00+10:00,0.0
3,ADP,ADPBA1,emissions_ADPBA1,2025-10-01T00:15:00+10:00,0.0
4,ADP,ADPBA1,emissions_ADPBA1,2025-10-01T00:20:00+10:00,0.0
...,...,...,...,...,...
6043,ADP,ADPBA1L,emissions_ADPBA1L,2025-10-07T21:20:00+10:00,0.0
6044,ADP,ADPBA1L,emissions_ADPBA1L,2025-10-07T21:45:00+10:00,0.0
6045,ADP,ADPBA1L,emissions_ADPBA1L,2025-10-07T21:50:00+10:00,0.0
6046,ADP,ADPBA1L,emissions_ADPBA1L,2025-10-07T22:00:00+10:00,0.0


The emission and power data of all facilities were retrieved. However, some facilities may have missing records due to the absence of data uploads during the specified time period, and thus they were not recorded in the CSV file. In addition, I assume that some facilities did not upload either power or emission data, and these facilities were also excluded from the dataset. This is because, as observed from the first few facilities such as ADP, even solar-based facilities still provide emission values (e.g., 0), indicating that the absence of such data likely means no data was uploaded rather than zero emissions.

In [179]:
def fetch_and_save_facility_data(
    operating_facilities,
    url,
    interval,
    date_start,
    date_end,
    output_csv="all_facilities_power.csv"
):
    """
    Fetch power and emission data for multiple facilities and save to a CSV.
    Any facility with missing data will be recorded in missing_facilities.

    Parameters:
        operating_facilities: list of dict, each dict must contain 'code' and 'name'
        url: str, API endpoint
        interval: str, data interval
        date_start: str or datetime, start date
        date_end: str or datetime, end date
        output_csv: str, CSV filename to save results

    Returns:
        missing_facilities: list of facility codes with missing data
    """
    missing_facilities = []

    # Loop through all operating facilities
    for facility in operating_facilities:
        fcode = facility['code']
        fname = facility['name']
        print(f"Fetching data for {fcode} ({fname})...")

        # Fetch power and emission data
        df_power = fetch_facility_metric(url, "power", interval, date_start, date_end, fcode)
        df_emission = fetch_facility_metric(url, "emissions", interval, date_start, date_end, fcode)

        # Skip if any data is missing
        if df_power.empty or df_emission.empty:
            print(f"Missing data for {fcode}, skipping...")
            missing_facilities.append(fcode)
            continue

        # Merge power and emission on facility_code, unit_code, timestamp
        df = pd.merge(df_power, df_emission, on=["facility_code", "unit_code", "timestamp"], how="outer")
        df.rename(columns={"value_x": "power", "value_y": "emission"}, inplace=True)
        df = df[['facility_code', 'unit_code', 'timestamp', 'power', 'emission']]

        # Save CSV (append if file exists)
        if os.path.exists(output_csv):
            df.to_csv(output_csv, mode="a", index=False, header=False)
        else:
            df.to_csv(output_csv, index=False)
        rows_saved = len(df)
        print(f"{fcode} data appended to {output_csv}, {rows_saved} rows saved.")

    print("All done.")
    print("Facilities with missing data:", missing_facilities)
    return missing_facilities


# call the function to fetch and save facility data
missing_facilities = fetch_and_save_facility_data(
    operating_facilities,
    url_Generation,
    interval,
    date_start,
    date_end,
    output_csv="all_facilities_power.csv"
)

Fetching data for ADP (Adelaide Desalination)...
ADP data appended to all_facilities_power.csv, 6048 rows saved.
Fetching data for ALDGASF (Aldoga)...
ALDGASF data appended to all_facilities_power.csv, 2016 rows saved.
Fetching data for ANGASTON (Angaston)...
ANGASTON data appended to all_facilities_power.csv, 2016 rows saved.
Fetching data for APPIN (Appin)...
Failed, status code: 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APPIN'] in the specified time range","success":false}
Failed, status code: 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APPIN'] in the specified time range","success":false}
Missing data for APPIN, skipping...
Fetching data for ARWF (Ararat)...
ARWF data appended to all_facilities_power.csv, 2016 rows saved.
Fetching data for AVLSF (Avonlie)...
AVLSF data appended to all_facilities_power.csv, 2016 rows saved.
Fetching data for DEIBDL (Bairnsdale)...
DEIBDL data appended to 

## Step 2:

In [180]:
def fetch_market_metric(url, metrics, interval, date_start, date_end, primary_grouping="network", print_mode = False, save_csv=False, csv_name="market_data.csv"):
    """
    Fetch market metric data from the OpenElectricity API for a specified interval and time range.

    Parameters:
        url (str): API endpoint URL.
        metrics (str): The metric to fetch, e.g., "price" or "demand".
        interval (str): Data interval, e.g., "5m", "1h", "1d".
        date_start (str or datetime): Start date for data fetching.
        date_end (str or datetime): End date for data fetching.
        primary_grouping (str, optional): Grouping for the API request. Defaults to "network".
        print_mode (bool, optional): Whether to print raw API response for debugging. Defaults to False.
        save_csv (bool, optional): Whether to save the resulting DataFrame to a CSV file. Defaults to False.
        csv_name (str, optional): Name of the CSV file to save if save_csv is True. Defaults to "market_data.csv".

    Returns:
        pd.DataFrame: DataFrame containing 'metric', 'timestamp', and 'value' columns.
    """

    # Set authorization header with API key
    headers = {"Authorization": f"Bearer {API_KEY}"}

    # Define request parameters
    params = {
        "metrics": metrics,
        "interval": interval,
        "date_start": date_start,
        "date_end": date_end,
        "primary_grouping": primary_grouping
    }

    # Make GET request to API
    response = requests.get(url, headers=headers, params=params)
    
    # Handle HTTP errors
    if response.status_code != 200:
        print(f"Failed, status code: {response.status_code}")
        print(response.text)
        return pd.DataFrame()

    # Parse data from JSON response
    data = response.json().get("data", [])
    
    # Handle case when no data is returned
    if not data:
        print("No data returned.")
        return pd.DataFrame()
    
    # Optionally print raw data for debugging
    if print_mode:
        print(data)
    
    # Transform JSON data into a flat list of rows
    rows = []
    for metric_block in data:
        results = metric_block.get("results", [])
        for entry in results:
            metric_name = entry["name"]
            for timestamp, value in entry["data"]:
                rows.append({
                    "metric": metric_name,
                    "timestamp": timestamp,
                    "value": value
                })

    # Convert list of rows into a Pandas DataFrame
    df = pd.DataFrame(rows)
    
    # Optionally save DataFrame to CSV
    if save_csv:
        df.to_csv(csv_name, index=False)
        print(f"Saved {len(df)} rows to {csv_name}")

    return df

In [181]:
# Test the market data fetch function for "price"
fetch_market_metric(
    url_market,
    metrics="price",
    interval=interval,
    date_start=date_start.isoformat(),
    date_end=date_end.isoformat(),
    primary_grouping="network",
    print_mode=True
)

[{'network_code': 'NEM', 'metric': 'price', 'unit': '$/MWh', 'interval': '5m', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'groupings': [], 'results': [{'name': 'price_total', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'columns': {}, 'data': [['2025-10-01T00:00:00+10:00', 25.796], ['2025-10-01T00:05:00+10:00', 29.506], ['2025-10-01T00:10:00+10:00', 26.118], ['2025-10-01T00:15:00+10:00', 34.272], ['2025-10-01T00:20:00+10:00', 29.218], ['2025-10-01T00:25:00+10:00', 27.264], ['2025-10-01T00:30:00+10:00', 26.718], ['2025-10-01T00:35:00+10:00', 25.904], ['2025-10-01T00:40:00+10:00', 26.696], ['2025-10-01T00:45:00+10:00', 26.11], ['2025-10-01T00:50:00+10:00', 28.712], ['2025-10-01T00:55:00+10:00', 25.964], ['2025-10-01T01:00:00+10:00', 25.89], ['2025-10-01T01:05:00+10:00', 26.968], ['2025-10-01T01:10:00+10:00', 30.086], ['2025-10-01T01:15:00+10:00', 24.842], ['2025-10-01T01:20:00+10:00', 25.464], ['2025-10-01T01

,metric,timestamp,value
0,price_total,2025-10-01T00:00:00+10:00,25.796
1,price_total,2025-10-01T00:05:00+10:00,29.506
2,price_total,2025-10-01T00:10:00+10:00,26.118
3,price_total,2025-10-01T00:15:00+10:00,34.272
4,price_total,2025-10-01T00:20:00+10:00,29.218
...,...,...,...
2011,price_total,2025-10-07T23:35:00+10:00,96.026
2012,price_total,2025-10-07T23:40:00+10:00,95.828
2013,price_total,2025-10-07T23:45:00+10:00,92.328
2014,price_total,2025-10-07T23:50:00+10:00,129.980


In [182]:
# Test the market data fetch function for "demand"
fetch_market_metric(
    url_market,
    metrics="demand",
    interval=interval,
    date_start=date_start.isoformat(),
    date_end=date_end.isoformat(),
    print_mode=True
)

[{'network_code': 'NEM', 'metric': 'demand', 'unit': 'MW', 'interval': '5m', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'groupings': [], 'results': [{'name': 'demand_total', 'date_start': '2025-10-01T00:00:00+10:00', 'date_end': '2025-10-07T23:55:00+10:00', 'columns': {}, 'data': [['2025-10-01T00:00:00+10:00', 20451.93], ['2025-10-01T00:05:00+10:00', 20443.37], ['2025-10-01T00:10:00+10:00', 20391.13], ['2025-10-01T00:15:00+10:00', 20348.11], ['2025-10-01T00:20:00+10:00', 20213.93], ['2025-10-01T00:25:00+10:00', 20116.28], ['2025-10-01T00:30:00+10:00', 20000.71], ['2025-10-01T00:35:00+10:00', 19904.92], ['2025-10-01T00:40:00+10:00', 19914.37], ['2025-10-01T00:45:00+10:00', 19851.06], ['2025-10-01T00:50:00+10:00', 19857.85], ['2025-10-01T00:55:00+10:00', 19726.98], ['2025-10-01T01:00:00+10:00', 19661.33], ['2025-10-01T01:05:00+10:00', 19614.56], ['2025-10-01T01:10:00+10:00', 19523.02], ['2025-10-01T01:15:00+10:00', 19392.47], ['2025-10-01T01:20:00

,metric,timestamp,value
0,demand_total,2025-10-01T00:00:00+10:00,20451.93
1,demand_total,2025-10-01T00:05:00+10:00,20443.37
2,demand_total,2025-10-01T00:10:00+10:00,20391.13
3,demand_total,2025-10-01T00:15:00+10:00,20348.11
4,demand_total,2025-10-01T00:20:00+10:00,20213.93
...,...,...,...
2011,demand_total,2025-10-07T23:35:00+10:00,20191.68
2012,demand_total,2025-10-07T23:40:00+10:00,20248.44
2013,demand_total,2025-10-07T23:45:00+10:00,20174.48
2014,demand_total,2025-10-07T23:50:00+10:00,20115.86


Get all market data for price and demand in the given time period. Missing value will be solve in next part.

In [183]:
def fetch_and_save_market_metrics(
    url_market,
    metrics_list,
    interval,
    date_start,
    date_end,
    output_csv="market.csv"
):
    """
    Fetch multiple market metrics (e.g., price, demand) and merge them into one CSV.

    Parameters:
        url_market (str): API endpoint for market data.
        metrics_list (list of str): List of metrics to fetch.
        interval (str): Data interval (e.g., "5m", "1h").
        date_start (str/datetime): Start date for fetching.
        date_end (str/datetime): End date for fetching.
        output_csv (str, optional): CSV file to save merged data. Defaults to "market.csv".

    Returns:
        None
    """

    all_dfs = []

    # Loop through each metric and fetch its data
    for metric in metrics_list:
        print(f"Fetching {metric} data...")
        df = fetch_market_metric(
            url_market,
            metrics=metric,
            interval=interval,
            date_start=date_start,
            date_end=date_end,
            print_mode=False
        )

        if df.empty:
            print(f"No data returned for {metric}, skipping...")
            continue

        # Rename the value column to the metric name
        df = df.rename(columns={"value": metric})

        # Keep only timestamp and metric columns
        df = df[["timestamp", metric]]

        all_dfs.append(df)

    if not all_dfs:
        print("No data fetched for any metric.")
        return

    # Merge all metrics on timestamp
    df_merged = all_dfs[0]
    for df in all_dfs[1:]:
        df_merged = pd.merge(df_merged, df, on="timestamp", how="outer")

    # Save to CSV
    if os.path.exists(output_csv):
        df_merged.to_csv(output_csv, mode="a", index=False, header=False)
    else:
        df_merged.to_csv(output_csv, index=False)

    print(f"All metrics saved to {output_csv}, {len(df_merged)} rows.")


# Call the function to fetch and save market metrics
fetch_and_save_market_metrics(
    url_market,
    ["price", "demand"],
    interval,
    date_start,
    date_end,
    output_csv="market.csv"
)


Fetching price data...
Fetching demand data...
All metrics saved to market.csv, 2016 rows.


# Part 2: Data Integration and Materialization/Caching 

## Step 0: Preprocessing all_facilities_power.csv

Since I have already merged all the data into a single CSV file named "all_facilities_power.csv", I now only need to perform data preprocessing.

In [184]:
# Read the CSV file and load it into a Pandas DataFrame
f_df = pd.read_csv("all_facilities_power.csv")

# Display the first five rows to quickly inspect the data structure and content
print(f_df.head())

  facility_code unit_code                  timestamp  power  emission
0           ADP    ADPBA1  2025-10-01T00:00:00+10:00 -0.004       0.0
1           ADP    ADPBA1  2025-10-01T00:05:00+10:00 -0.046       0.0
2           ADP    ADPBA1  2025-10-01T00:10:00+10:00  0.000       0.0
3           ADP    ADPBA1  2025-10-01T00:15:00+10:00  0.003       0.0
4           ADP    ADPBA1  2025-10-01T00:20:00+10:00 -0.018       0.0


The negative values in the power column are primarily due to units that are batteries. Unlike conventional generators, batteries can both charge and discharge. When a battery is charging, it draws power from the grid, which is recorded as a negative value. Conversely, when it discharges, it supplies power to the grid, recorded as a positive value. Therefore, negative power values in this dataset reflect normal operation for battery units.

In [185]:
f_df.isnull().sum()

facility_code    0
unit_code        0
timestamp        0
power            0
emission         0
dtype: int64

As observed, the dataset contains no missing values, as they were handled during the data retrieval process. Facilities without available power or emission data were excluded from the dataset. Additionally, missing values were not imputed with zeros, since this would alter the semantic meaning of the data.

In [ ]:
# facilities who miss data or power data; I record them when I retrieve data from API
print("There are", len(missing_facilities), "facilities who miss data or power data")
missing_facilities # If you did not run Part 1, you may see an empty list here or get an error.

There are 84 facilities who miss data or power data


['APPIN',
 'BAKING',
 'BAPS',
 'BERWICK',
 'BLAYNEY',
 'BROOKLYN',
 'BROWNMT',
 'BPLANDF',
 'BDONGHYD',
 'BURRIN',
 'CESF',
 'CHPSTWF',
 'CLAYTON',
 'CODRGTON',
 'CBWF',
 'COPTNHYD',
 'CROOKWEL',
 'EASTCRK',
 'EASTCRK2',
 'EILDONPD',
 'FWF',
 'GLBWNHYD',
 'GLENMAGG',
 'GLENNCRK',
 'GRANGEAV',
 'GROSV1',
 'GROSV2',
 'HLMSEW01',
 'HALAMRD',
 'HAMPTON',
 'HASTING',
 'HEPWIND',
 'INVICTA',
 'ICSM',
 'JACKSGUL',
 'JNDSHPS',
 'JOUNAMA',
 'KEEPIT',
 'AGLSITA',
 'LIMBESS',
 'LONGFORD',
 'LUCASHGT',
 'LUCAS2S2',
 'MACKAY',
 'MAROOWF1',
 'MIDLDPS',
 'MORANBAH',
 'NASF',
 'OCPS',
 'OAKY2',
 'PINDARI',
 'RACOMIL',
 'SHEP1',
 'SKSF',
 'STGEORGE',
 'STAPYLTON',
 'VALDORA',
 'TABMILL2',
 'TAHMOOR',
 'TATIARA',
 'TATURA',
 'THEDROP',
 'TIMWEST',
 'TOORAWF',
 'TOWER',
 'TGNSS',
 'TITREE',
 'VICMILL',
 'PRDSO_WALPOLE',
 'WESTILLA',
 'WHIT1',
 'WILGAPK',
 'WILGAPKB',
 'WILLHOV',
 'WINDHILL',
 'WINGF1',
 'WINGF2',
 'WOLLERT',
 'WONWP',
 'WDLNGN',
 'WYANGALA',
 'WYANGALB',
 'YWNGAHYD',
 'YAWWF']

## Step 1: Preprocessing "market.csv"

In [187]:
# Read the CSV file containing market data and load it into a Pandas DataFrame
m_df = pd.read_csv("market.csv")

# Display the first five rows to quickly inspect the data structure and content
print(m_df.head())

# Get the total number of rows in the DataFrame
m_df.shape[0]

                   timestamp   price    demand
0  2025-10-01T00:00:00+10:00  25.796  20451.93
1  2025-10-01T00:05:00+10:00  29.506  20443.37
2  2025-10-01T00:10:00+10:00  26.118  20391.13
3  2025-10-01T00:15:00+10:00  34.272  20348.11
4  2025-10-01T00:20:00+10:00  29.218  20213.93


2016

In [188]:
m_df.isnull().sum()

timestamp     0
price        41
demand        4
dtype: int64

As you may notice, there are 41 missing values in the price column and 4 missing values in the demand column. The dataset contains 2,016 rows in total. I have decided to remove these rows directly, because the missing values cannot be reliably estimated from other data. Additionally, filling the missing values with 0 could lead to misinterpretation, so deletion is the safest approach.

In [189]:
m_df = m_df.dropna()
m_df.isnull().sum()

timestamp    0
price        0
demand       0
dtype: int64

## Step 2: Preprocessing nem.operating_facilities.csv

This csv file store all operating facilities in NEM. Our rest part will use this table as a dictionary to search location or units' fuel technology, so I need to do some cleaning to make it easier to use.

In [216]:
all_fc = pd.read_csv("nem_operating_facilities.csv")
all_fc.head()

,code,name,network_id,network_region,description,units,updated_at,created_at,location.lat,location.lng
0,ADP,Adelaide Desalination,NEM,SA1,"<p>The Adelaide Desalination plant (ADP), form...","[{'code': 'ADPPV1', 'fueltech_id': 'solar_util...",2025-08-05T06:08:12Z,2023-10-18T04:34:30Z,-35.096948,138.484061
1,ALDGASF,Aldoga,NEM,QLD1,<p>The Aldoga Solar Farm will be approximately...,"[{'code': 'ALDGASF1', 'fueltech_id': 'solar_ut...",2025-03-25T00:52:44Z,2025-01-31T04:19:33Z,-23.839544,151.084900
2,AMCORGR,Amcor Glass,NEM,SA1,<p></p>,"[{'code': 'AMCORGR', 'fueltech_id': 'distillat...",2023-10-18T04:34:32Z,2023-10-18T04:34:32Z,-34.882663,138.577975
3,ANGASTON,Angaston,NEM,SA1,<p>Angaston Power Station is a diesel-powered ...,"[{'code': 'ANGAS2', 'fueltech_id': 'distillate...",2025-09-07T01:53:13Z,2023-10-18T04:34:32Z,-34.503948,139.024296
4,APS,Anglesea,NEM,VIC1,<p>The Anglesea Power Station was a brown coal...,"[{'code': 'APS', 'fueltech_id': 'coal_brown', ...",2024-11-04T00:41:34Z,2023-10-18T04:34:32Z,-38.389031,144.180589


After discussion, we decided to retain only the columns code, name, network_id, network_region, location.lat, and location.lng from the dataset, as these fields are sufficient for identifying and locating each facility. The columns description, updated_at, and created_at were removed because they do not contribute to our analysis and only increase data redundancy. The units field will be cleaned separately in the next step.

In [217]:
all_fc.drop(columns=["description", "updated_at", "created_at"], inplace=True)
all_fc.head()

,code,name,network_id,network_region,units,location.lat,location.lng
0,ADP,Adelaide Desalination,NEM,SA1,"[{'code': 'ADPPV1', 'fueltech_id': 'solar_util...",-35.096948,138.484061
1,ALDGASF,Aldoga,NEM,QLD1,"[{'code': 'ALDGASF1', 'fueltech_id': 'solar_ut...",-23.839544,151.084900
2,AMCORGR,Amcor Glass,NEM,SA1,"[{'code': 'AMCORGR', 'fueltech_id': 'distillat...",-34.882663,138.577975
3,ANGASTON,Angaston,NEM,SA1,"[{'code': 'ANGAS2', 'fueltech_id': 'distillate...",-34.503948,139.024296
4,APS,Anglesea,NEM,VIC1,"[{'code': 'APS', 'fueltech_id': 'coal_brown', ...",-38.389031,144.180589


Next, within the units column, each entry contains multiple pieces of unnecessary information. We only need a simplified dictionary that maps each unit code to its corresponding fuel technology. This allows us to better understand the composition of each facility.

Additionally, for visualization purposes in Part 4, we plan to categorize similar fuel types under broader groups. For instance, all fuel technologies starting with gas_ will be classified under the “gas” category, and similar aggregation will be performed for other fuel types as well. This categorization helps make the analysis clearer and the visualizations more interpretable. Detail should be {'battery': 'Battery', 'battery_charging': 'Battery', 'battery_discharging': 'Battery', 'bioenergy_biogas': 'Bioenergy', 'bioenergy_biomass': 'Bioenergy', 'coal_black': 'Coal', 'coal_brown': 'Coal', 'distillate': 'distillate', 'gas_ccgt': 'Gas', 'gas_ocgt': 'Gas', 'gas_recip': 'Gas', 'gas_steam': 'Gas', 'gas_wcmg': 'Gas', 'hydro': 'Hydro', 'pumps': 'Hydro', 'solar_utility': 'Solar', 'wind': 'Wind'}.

In [218]:
# Assume the DataFrame is named all_fc
fueltech_types = set()  # Use a set to store unique fueltech types

for units in all_fc["units"]:
    if pd.isna(units):
        continue
    # Convert string representation of list back to Python object (if read from CSV)
    if isinstance(units, str):
        units = ast.literal_eval(units)
    
    # Iterate through each unit and extract fueltech_id
    for u in units:
        fueltech_id = u.get("fueltech_id")
        if fueltech_id:
            fueltech_types.add(fueltech_id)

# Convert to a sorted list for easier viewing
fueltech_list = sorted(list(fueltech_types))

print(f"Total unique fueltech types: {len(fueltech_list)}")
print(fueltech_list)

# Automatically generate a many-to-one mapping based on prefixes
fueltech_map = {}

for f in fueltech_list:
    if f.startswith("battery"):
        fueltech_map[f] = "Battery"
    elif f.startswith("bioenergy"):
        fueltech_map[f] = "Bioenergy"
    elif f.startswith("coal"):
        fueltech_map[f] = "Coal"
    elif f.startswith("gas"):
        fueltech_map[f] = "Gas"
    elif f.startswith("solar"):
        fueltech_map[f] = "Solar"
    elif f.startswith("wind"):
        fueltech_map[f] = "Wind"
    elif f.startswith("pump"):
        fueltech_map[f] = "Hydro"  # Pumps are usually categorized as hydro
    elif f.startswith("hydro"):
        fueltech_map[f] = "Hydro"
    else:
        # Keep original name if it doesn't match any prefix
        fueltech_map[f] = f

print(fueltech_map)


Total unique fueltech types: 17
['battery', 'battery_charging', 'battery_discharging', 'bioenergy_biogas', 'bioenergy_biomass', 'coal_black', 'coal_brown', 'distillate', 'gas_ccgt', 'gas_ocgt', 'gas_recip', 'gas_steam', 'gas_wcmg', 'hydro', 'pumps', 'solar_utility', 'wind']
{'battery': 'Battery', 'battery_charging': 'Battery', 'battery_discharging': 'Battery', 'bioenergy_biogas': 'Bioenergy', 'bioenergy_biomass': 'Bioenergy', 'coal_black': 'Coal', 'coal_brown': 'Coal', 'distillate': 'distillate', 'gas_ccgt': 'Gas', 'gas_ocgt': 'Gas', 'gas_recip': 'Gas', 'gas_steam': 'Gas', 'gas_wcmg': 'Gas', 'hydro': 'Hydro', 'pumps': 'Hydro', 'solar_utility': 'Solar', 'wind': 'Wind'}


With the simplified category dict, we can now preprocess the units column.

In [219]:
def simplify_units(units, fueltech_map):
    """
    Convert the list of unit dictionaries into a simplified mapping:
    {unit_code: mapped_fueltech}

    If the value in 'fueltech_id' matches a key in fueltech_map,
    replace it with the corresponding general category.
    """
    if pd.isna(units):
        return None

    # Convert from string to Python object if necessary
    if isinstance(units, str):
        try:
            units = ast.literal_eval(units)
        except Exception:
            return None

    result = {}
    for u in units:
        unit_code = u.get("code")
        fueltech_id = u.get("fueltech_id")

        if unit_code and fueltech_id:
            # Map detailed type to general category
            mapped_type = fueltech_map.get(fueltech_id, fueltech_id)
            result[unit_code] = mapped_type

    return result

# Apply the function to the DataFrame
all_fc["units"] = all_fc["units"].apply(simplify_units, args=(fueltech_map,))

# Remove the last character from each value in 'network_region' column
all_fc["network_region"] = all_fc["network_region"].str[:-1]

# Preview result
all_fc.head()

# Save the processed DataFrame to CSV
all_fc.to_csv("nem_fc_dict.csv", index=False)

print("Saved all_fc to nem_fc_dict.csv")

Saved all_fc to nem_fc_dict.csv


# Part 3: Data Publishing via MQTT and Continuous Execution

In [3]:
# --- Import MQTT and Required Libraries ---
import paho.mqtt.client as mqtt
import pandas as pd
import time
import json
import ast  # For safely evaluating string representations of dictionaries

# --- MQTT Configuration ---
# Public MQTT broker configuration for accessibility
BROKER_HOST = "broker.hivemq.com"
BROKER_PORT = 1883

# Unique topic identifier to prevent data collision between different users
UNIQUE_TOPIC = "comp5339/a2/5339_tut22_group08" 

# --- Data Loading and Validation ---
# Load pre-processed power data and facility metadata
try:
    # Load real-time power generation data from cache
    df_stream = pd.read_csv("all_facilities_power.csv")
    print(f"Successfully loaded all_facilities_power.csv, total {len(df_stream)} records.")

    # Load facility code to name mapping dictionary
    df_facility_dict = pd.read_csv("nem_fc_dict.csv")
    print(f"Successfully loaded nem_fc_dict.csv, total {len(df_facility_dict)} power plants.")
    
except FileNotFoundError as e:
    print(f"Error: Required file not found {e.filename}. Please execute Part 1 & 2 preprocessing steps.")
    raise e

# --- Data Stream Preparation ---
# Sort data by timestamp to maintain chronological order in streaming
print("Sorting data stream by timestamp...")
df_stream_sorted = df_stream.sort_values(by="timestamp").reset_index(drop=True)
print("Data sorting completed.")


# --- MQTT Client Configuration ---
def on_connect(client, userdata, flags, rc, properties=None):
    """Callback function for MQTT connection events"""
    if rc == 0:
        print(f"Successfully connected to MQTT Broker: {BROKER_HOST}")
    else:
        print(f"MQTT connection failed, return code: {rc}")

def on_publish(client, userdata, mid, rc, properties=None):
    """Callback function for message publication events"""
    # Error handling for failed message publications
    if rc != 0:
        print(f"Message publication failed (Message ID: {mid}), return code: {rc}")

# Initialize MQTT client with version 2 callback API
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.on_connect = on_connect
client.on_publish = on_publish

# Establish connection to MQTT broker
print(f"Establishing connection to {BROKER_HOST}...")
client.connect(BROKER_HOST, BROKER_PORT, 60)

# Start background thread for network traffic handling
client.loop_start() 

# --- Continuous Data Streaming Implementation ---
print("--- MQTT Data Publisher initialized ---")

try:
    while True:
        print(f"\n--- Starting new data stream cycle ---")
        
        # --- Message Publication Loop ---
        # Iterate through sorted dataset and publish each record
        for row in df_stream_sorted.itertuples():
            
            # Construct message payload with facility data
            payload = {
                "facility_code": row.facility_code,
                "unit_code": row.unit_code,
                "timestamp": row.timestamp,
                "power": row.power,
                "emission": row.emission
            }
            
            # Serialize payload to JSON format for transmission
            message = json.dumps(payload)
            
            # Publish message with retain flag for persistent storage
            result = client.publish(UNIQUE_TOPIC, message, retain=True)
            
            # Validate publication status
            if result.rc != 0:
                print(f"Message publication error: {result}")

            # Throttle message rate to simulate real-time data
            time.sleep(0.1) 

        # --- Stream Cycle Management ---
        print(f"--- Completed publication of {len(df_stream_sorted)} messages ---")
        print(f"--- Entering inter-cycle delay period ---")
        time.sleep(60)

except KeyboardInterrupt:
    # Handle user-initiated termination
    print("\n--- Data streaming terminated by user ---")

finally:
    # --- Resource Cleanup ---
    print("Initiating MQTT client shutdown...")
    client.loop_stop()
    client.disconnect()
    print("MQTT client resources released.")

Successfully loaded all_facilities_power.csv, total 979365 records.
Successfully loaded nem_fc_dict.csv, total 514 power plants.
Sorting data stream by timestamp...
Data sorting completed.
Establishing connection to broker.hivemq.com...
--- MQTT Data Publisher initialized ---

--- Starting new data stream cycle ---
Successfully connected to MQTT Broker: broker.hivemq.com

--- Data streaming terminated by user ---
Initiating MQTT client shutdown...
MQTT client resources released.
